In [1]:
# Setup movie data preprocessing environment
import sys
import pandas as pd
import numpy as np
from IPython.display import display

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from collecting_user_data import User

In [11]:
# Ingest cleaned movie data
movies = pd.read_csv("../data/movies.csv")

# Ingest ratings csv file to gather user ratings of movies
ratings = pd.read_csv("../data/ratings_small.csv")

In [3]:
display(movies)

,genres,id,overview,popularity,poster_path,release_date,runtime,title,vote_average,keywords,soup
0,"['animation', 'comedy', 'family']",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",jealousy toy boy friendship friends rivalry bo...
1,"['adventure', 'fantasy', 'family']",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,104.0,Jumanji,6.9,"['boardgame', 'disappearance', ""basedonchildre...",boardgame disappearance basedonchildren'sbook ...
2,"['romance', 'comedy']",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,101.0,Grumpier Old Men,6.5,"['fishing', 'bestfriend', 'duringcreditsstinge...",fishing bestfriend duringcreditsstinger oldmen...
3,"['comedy', 'drama', 'romance']",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,127.0,Waiting to Exhale,6.1,"['basedonnovel', 'interracialrelationship', 's...",basedonnovel interracialrelationship singlemot...
4,['comedy'],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,106.0,Father of the Bride Part II,5.7,"['baby', 'midlifecrisis', 'confidence', 'aging...",baby midlifecrisis confidence aging daughter m...
...,...,...,...,...,...,...,...,...,...,...,...
23257,"['fantasy', 'comedy']",49271,A man rents an apartment and furnishes it in r...,0.725084,/tX3x4Jms6xqVgiet0Ei2Ks0XkAg.jpg,1909-01-01,6.0,The Devilish Tenant,6.7,"['tenant', 'silentfilm']",tenant silentfilm fantasy comedy
23258,"['animation', 'family']",14885,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,/yU6byKhpzHG1RFpY5PYM6mfhfU1.jpg,2005-09-13,67.0,Pooh's Heffalump Halloween Movie,5.4,"['halloween', 'winniethepooh']",halloween winniethepooh animation family
23259,"['horror', 'mystery', 'thriller']",84419,An unsuccessful sculptor saves a madman named ...,0.222814,/yMnq9mL5uYxbRgwKqyz1cVGCJYJ.jpg,1946-03-29,65.0,House of Horrors,6.3,"['revenge', 'murder', 'serialkiller', 'newyork...",revenge murder serialkiller newyorkcity sculpt...
23260,"['mystery', 'horror']",390959,"In this true-crime documentary, we delve into ...",0.076061,/q75tCM4pFmUzdCg0gqcOQquCaYf.jpg,2000-10-22,45.0,Shadow of the Blair Witch,7.0,['blairwitch'],blairwitch mystery horror


In [13]:
# Grabbing the inputs from the user
user = User()
user.get_searchTerms()
user.set_fav_movie()

In [14]:
print("User search terms: ", user.search_terms)
print("User favorite movie: ", user.fav_movie)

User search terms:  ['comedy romance', 'fun light guns']
User favorite movie:  blindside


In [16]:
def make_recommendation(movies = movies, search_terms = [], fav_movie = None):
  # Creating a copy of the last row of the dataset, which we will use to input the user's input
  new_row = movies.iloc[-1,:].copy()

  # Adding the input to the new row
  new_row.iloc[-1] = " ".join(search_terms)
  
  # Adding the new row to the dataset
  movies = movies.append(new_row)
  
  # Vectorizing the entire matrix
  count = CountVectorizer(stop_words='english')
  count_matrix = count.fit_transform(movies['soup'])

  # Pairwise cosine similarity and creating the cosine matrix
  cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
  
  # Sorting cosine similarities by highest to lowest
  sim_scores = list(enumerate(cosine_sim2[-1,:]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_idx = [i[0] for i in sim_scores]

  if fav_movie:
    # Matching the similarities to the movie titles and vote average while incorporating collaborative filtering as well (hybrid approach)
    ranked_titles = []
    ranked_movies = movies.iloc[movie_idx][['title', 'vote_count', 'vote_average', 'year', 'id']]
    ranked_movies['score'] = ranked_movies.id.apply(lambda x: svd.predict())
  
  if not fav_movie:
  # Matching the similarities to the movie titles and vote_average (content-based recs only)
    ranked_titles = []
    for i in range(10):
      indx = sim_scores[i][0]
      ranked_titles.append([movies['title'].iloc[indx], movies['id'].iloc[indx]])
  
  return ranked_titles

make_recommendation(movies, user.search_terms, user.fav_movie)

/var/folders/04/382s5ryn4cg90lmcyzj298g00000gn/T/ipykernel_18251/2564834425.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies = movies.append(new_row)


,genres,id,overview,popularity,poster_path,release_date,runtime,title,vote_average,keywords,soup
0,"['animation', 'comedy', 'family']",862,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,81.0,Toy Story,7.7,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",jealousy toy boy friendship friends rivalry bo...
1,"['adventure', 'fantasy', 'family']",8844,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,104.0,Jumanji,6.9,"['boardgame', 'disappearance', ""basedonchildre...",boardgame disappearance basedonchildren'sbook ...
2,"['romance', 'comedy']",15602,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,101.0,Grumpier Old Men,6.5,"['fishing', 'bestfriend', 'duringcreditsstinge...",fishing bestfriend duringcreditsstinger oldmen...
3,"['comedy', 'drama', 'romance']",31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,127.0,Waiting to Exhale,6.1,"['basedonnovel', 'interracialrelationship', 's...",basedonnovel interracialrelationship singlemot...
4,['comedy'],11862,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,106.0,Father of the Bride Part II,5.7,"['baby', 'midlifecrisis', 'confidence', 'aging...",baby midlifecrisis confidence aging daughter m...
...,...,...,...,...,...,...,...,...,...,...,...
23258,"['animation', 'family']",14885,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,/yU6byKhpzHG1RFpY5PYM6mfhfU1.jpg,2005-09-13,67.0,Pooh's Heffalump Halloween Movie,5.4,"['halloween', 'winniethepooh']",halloween winniethepooh animation family
23259,"['horror', 'mystery', 'thriller']",84419,An unsuccessful sculptor saves a madman named ...,0.222814,/yMnq9mL5uYxbRgwKqyz1cVGCJYJ.jpg,1946-03-29,65.0,House of Horrors,6.3,"['revenge', 'murder', 'serialkiller', 'newyork...",revenge murder serialkiller newyorkcity sculpt...
23260,"['mystery', 'horror']",390959,"In this true-crime documentary, we delve into ...",0.076061,/q75tCM4pFmUzdCg0gqcOQquCaYf.jpg,2000-10-22,45.0,Shadow of the Blair Witch,7.0,['blairwitch'],blairwitch mystery horror
23261,['horror'],289923,A film archivist revisits the story of Rustin ...,0.386450,/lXtoHVdej6kS1Dc7KAhw05sMos9.jpg,2000-10-03,30.0,The Burkittsville 7,7.0,"['witch', 'mythology', 'legend', 'serialkiller...",witch mythology legend serialkiller mockumenta...


In [20]:
display(movies[movies.title == "The Blind Side"])

,genres,id,overview,popularity,poster_path,release_date,runtime,title,vote_average,keywords,soup
10025,['drama'],22881,"Oversized African-American, Michael Oher, the ...",12.753109,/wEGFSgPebLk6g1ngKsJ6a46PYLV.jpg,2009-11-20,129.0,The Blind Side,7.2,"['americanfootball', 'adoption', 'education', ...",americanfootball adoption education privatesch...


In [23]:
ratings[ratings.movieId == 31]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
498,7,31,3.0,851868750
6059,31,31,4.0,1273541953
6130,32,31,4.0,834828440
6526,36,31,3.0,847057202
6773,39,31,3.0,832525157
10223,73,31,3.5,1255591860
13516,88,31,3.0,1239755559
14810,96,31,2.5,1223256331
16867,110,31,4.0,840100695
